In [6]:
from data_generator import DataGenerator
import config
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'config'

In [ ]:
data_size = len(config.train_set)
list_IDs = np.arange(data_size)
np.random.shuffle(list_IDs)
val_start = int(data_size*.95)
batch_size = 128

# Generators
train_gen = DataGenerator(list_IDs[val_start:], config.train_set,
                          config.index_map, batch_size=batch_size)
val_gen = DataGenerator(list_IDs[:val_start], config.train_set,
                        config.index_map, batch_size=batch_size)

input_shape = (160, 85, 3)
model = tf.keras.applications.EfficientNetB0(
    weights=None,
    input_shape=input_shape,
    classes=250
)
model.compile(optimizer='Adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

# checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5',
                                                monitor='accuracy', mode='max',
                                                verbose=1, save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                      verbose=1, patience=200)
callbacks_list = [es, checkpoint]

# Train model on dataset
model.fit(x=train_gen,
          validation_data=val_gen,
          epochs=20,
          callbacks=callbacks_list)